# Performing a Flythrough in a Notebook
This notebook tutorial shows how to perform a flythrough in a notebook in a variety of methods.
1. Fly a real satellite trajectory through a dataset,
2. Fly a sample trajectory through a dataset,
3. Fly a two-line elements (TLEs) through a dataset,
4. Fly a trajectory from a file through a dataset,
5. Fly a trajectory in python memory through a dataset.

Note: You may run the notebook as is if you have the sample data files, but you must
change the 'file_dir', 'output_name', and 'plot_output' variables the relevant blocks to have the correct file path. The sample data files can be obtained from https://ccmc.gsfc.nasa.gov/RoR_WWW/output_files/KAMODO_DEMO/GITM/jasoon_shim_071418_IT_1_subset/. Only the \*.nc files need to be downloaded.

All coordinate conversions needed are automatically detected and handled in the background. These routines are designed for scalars. Proper treatment of vectors in this process is a capability currently under development by Ensemble's Kamodo-core.

In [2]:
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF

## RealFlight
Fly a real satellite trajectory obtained from the SSCWeb through a dataset.

In [2]:
help(SF.RealFlight)

Help on function RealFlight in module kamodo_ccmc.flythrough.SatelliteFlythrough:

RealFlight(dataset, start, stop, model, file_dir, variable_list, coord_type='GEO', output_name='', plot_coord='GEO', verbose=False)
    Retrieves the trajectory for the satellite requested and then flies that
    trajectory through the model data requested.
    
    dataset: name of the satellite data set to pull trajectory from
    start: utc timestamp for start of desired time interval
    stop: utc timestamp for end of desired time interval
    model: 'CTIPe','IRI', ...
    file_dir: complete path to where model data files are stored
    variable_list: List of standardized variable names. See model variable
        output for details. Dimensions must be at least time + 2D spatial.
    coord_type: Pick from GEO, GSM, GSE, or SM for the satellite trajectory.
    output_name: complete path with filename (with the extension) for the file
        to write the results to. Plotting filenames are determined b

In [3]:
# Select input values
import datetime as dt
# Choose input values for RealFlight function call.
model = 'GITM'  # Choose the model.
# Full file path to model output data.
file_dir = 'D:/GITM/jasoon_shim_071418_IT_1_tenth_oneday/'  
start_utcts = dt.datetime(2015, 3, 17, 0).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2015, 3, 17, 2).replace(tzinfo=dt.timezone.utc).timestamp()-1
variables = ['rho_n', 'TEC']  # one or more variable names to retrieve
dataset = 'cnofs' 
# Use https://sscweb.gsfc.nasa.gov/ to find the satellite name and time range desired.
coord_type = 'GEO'  # Desired coordinate system for retrieved trajectory.
# Choose naming convention for output files
output_name = 'Files/RealFlightExample_GITM.txt' 
plot_coord = 'GSE'  # Coordinate system chosen for output plots
# See https://sscweb.gsfc.nasa.gov/users_guide/Appendix_C.shtml for a description of coordinate types

The flythrough commands will break if the output file requested already exists. This feature is included by design to prevent users from accidentally overwriting a previously created file, especially since some flythrough execution times may take a while depending on various factors. So, the next block removes the file if it already exists. Avoid executing the next block if this is not the desired behavior.

In [4]:
# Remove the previously created output file.
import os
if os.path.isfile(output_name):
    os.remove(output_name)
    print(output_name, 'file removed.')
else:
    print(output_name, 'file not found.')

Files/RealFlightExample_GITM.txt file removed.


In [5]:
# Run RealFlight flythrough command. 
results_real = SF.RealFlight(dataset, start_utcts, end_utcts, model, file_dir, variables, 
                             coord_type=coord_type, output_name=output_name, plot_coord=plot_coord)
# The numerous 'Time slice index' messages printed are generated by the lazy time interpolation,
# which only loads the time slices necessary to perform the flythrough execution requested.
# Open plots in separate internet browser window for interactivity. Nothing will open here.

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 added from file.
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice 

C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\scipy\interpolate\interpolate.py:630: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\scipy\interpolate\interpolate.py:633: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


-saving html div file:  Files/RealFlightExample_GITM_rho_n_3D.html
-saving html div file:  Files/RealFlightExample_GITM_rho_n_1D.html
-saving html div file:  Files/RealFlightExample_GITM_TEC_3D.html
-saving html div file:  Files/RealFlightExample_GITM_TEC_1D.html


In [6]:
# Functionalize the output.
kamodo_object_real = SF.O.Functionalize_SFResults(model, results_real)
kamodo_object_real

{rho_n(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C460A6C9D8>, rho_n: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C460A6C9D8>, TEC(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C4609CA828>, TEC: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C4609CA828>}

## FakeFlight
Fly a sample satellite trajectory through a dataset

In [7]:
help(SF.FakeFlight)

Help on function FakeFlight in module kamodo_ccmc.flythrough.SatelliteFlythrough:

FakeFlight(start_time, stop_time, model, file_dir, variable_list, max_lat=65.0, min_lat=-65.0, lon_perorbit=363.0, max_height=450.0, min_height=400.0, p=0.01, n=2.0, verbose=False, output_name='', plot_coord='GEO')
    Generates a sample trajectory and then flies that trajectory through the
    model data chosen.
    
    Parameters:
        start_time: utc timestamp in seconds for start
        stop_time: utc timestamp in seconds for stop
        model: CTIPe, IRI, ....
        file_dir: complete path to where model data is stored
        variable_list: list of standardized variable names desired.
            Integers allowed. Dimensions must be at least time + 2D spatial.
        max_lat: maximum latitude for sample trajectory, in degrees
            (default=65.)
        min_lat: minimum latitude for sample trajectory, in degrees
            (default=-65.)
        lon_perorbit: the degrees of longitud

In [8]:
# Run FakeFlight. Note the same variables defined above are used here. 
results_fake = SF.FakeFlight(start_utcts, end_utcts, model, file_dir, variables, max_height=400, min_height=300, n=20)

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])
(c1,c2,c3) = (lon, lat, alt) in (deg,deg,km) in the GDZ, sph coordinate system.
sat_time contains the utc timestamps.
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 added from file.
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Ti

C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\scipy\interpolate\interpolate.py:630: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\scipy\interpolate\interpolate.py:633: RuntimeWarning:

invalid value encountered in multiply



In [9]:
# Functionalize the output.
kamodo_object_fake = SF.O.Functionalize_SFResults(model, results_fake)
kamodo_object_fake

{rho_n(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C4609D1948>, rho_n: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C4609D1948>, TEC(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C4609D14C8>, TEC: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C4609D14C8>}

## TLEFlight
The TLEFlight function uses two-line elements to calculate a satellite trajectory using sgp4, 
and then flies that satellite trajectory through the chosen model data.

In [10]:
help(SF.TLEFlight)

Help on function TLEFlight in module kamodo_ccmc.flythrough.SatelliteFlythrough:

TLEFlight(tle_file, start, stop, time_cadence, model, file_dir, variable_list, output_name='', plot_coord='GEO', method='forward', verbose=False)
    Use sgp4 to calculate a satellite trajectory given TLEs, then fly the
    trajectory through the chosen model data. If the time cadence does not
    evenly divide into the range of timestamps given, then the ending time
    value will be extended so that the entire requested range will be covered.
    Parameters:
        tle_file: The file name, including complete file path, of a file
            containing two-line elements. It is assumed that the file has no
            header and no other content.
        start_utcts: The UTC timestamp corresponding to the desired start time.
            Should be an integer.
        stop_utcts: The UTC timestamp corresponding to the desired stop time.
            Should be an integer.
        time_cadence: The number of 

In [11]:
# Run TLEFlight flythrough command.
tle_file = 'Files/GRACE1_TLEs.txt'
time_cadence = 60.  # seconds between propagated trajectory positions
results_tle = SF.TLEFlight(tle_file, start_utcts, end_utcts, time_cadence, model, file_dir, variables)

Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 added from file.
Time slice index 14 added from file.
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 a

In [12]:
# Functionalize the output.
kamodo_object_tle = SF.O.Functionalize_SFResults(model, results_tle)
kamodo_object_tle

{rho_n(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461C51A68>, rho_n: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461C51A68>, TEC(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461CD8828>, TEC: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461CD8828>}

## MyFlight
The MyFlight function flies a trajectory from a file through the chosen model data. The file must be formatted to match the csv/nc/txt file examples given. The easiest way to create this is to write out a given trajectory to a file using the SF.O.SF_write function demonstrated in the Trajectories section and use the output file from there.

In [13]:
help(SF.MyFlight)

Help on function MyFlight in module kamodo_ccmc.flythrough.SatelliteFlythrough:

MyFlight(traj_file, model, file_dir, variable_list, output_name='', plot_coord='GEO', verbose=False)
    Read in a trajectory from a file, then fly through the model data
    selected.
    
    traj_file: complete path and filename for file containing trajectory data.
    file_type: one of 'cdf4' for netCDF4 files, 'csv' for comma-separated
        files, or 'txt' for a tab-separated text file. Indicates the format of
        the input trajectory file.
    coord_sys: one of 'GDZ', 'GEO', 'GSM', 'GSE', 'SM', 'GEI', 'MAG', 'SPH',
        'RLL' combined with '-sph' or '-car'. E.g. 'SM-car' or 'GDZ-sph'.
        Astropy coordinate systems also supported.
    model: 'CTIPe', 'IRI', ...
    file_dir: complete path to model data files
    variable_list: List of standardized variable names. See model variable
        output for details. Dimensions must be at least time + 2D spatial.
    output_name: complete path 

In [14]:
# Run MyFlight with user-supplied trajectory
traj_file = 'Files/RealFlightExample_GITM.txt' 
results_my = SF.MyFlight(traj_file, model, file_dir, variables)

Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 added from file.
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 a

In [15]:
# Functionalize the output.
kamodo_object_my = SF.O.Functionalize_SFResults(model, results_my)
kamodo_object_my

{rho_n(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461DDC828>, rho_n: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461DDC828>, TEC(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461DDC948>, TEC: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C461DDC948>}

## ModelFlythrough
The ModelFlythough function flies a user-supplied trajectory through the chosen model data. This is the function the other flythrough functions call once a trajectory is acquired. The blocks below demonstrate this by first getting a trajectory from one of the Kamodo trajectory functions - SatelliteTrajectory in this case - and can be done with any of the trajectory functions. See the Trajectory section for more details.

In [16]:
help(SF.ModelFlythrough)

Help on function ModelFlythrough in module kamodo_ccmc.flythrough.SatelliteFlythrough:

ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, coord_sys, output_name='', plot_coord='GEO', verbose=False, _print_units=True)
    Call satellite flythrough wrapper specific to the model chosen.
    Parameters:
    ------------
    model: 'CTIPe','IRI', ...
    file_dir: complete path to where model data files are stored
    variable_list: List of standardized variable names. See model variable
        output for details. Dimensions must be at least time + 2D spatial.
    sat_time: a numpy array of the utc timestamps
    c1, c2, c3: numpy arrays of the positions correlating to the utc timestamps
        (c1, c2, c3) should be (x,y,z) in R_E for cartesian coordinates, and
        (lon, lat, radius (R_E) or altitude (km)) for spherical coordinates.
    coord_sys: one of 'GDZ', 'GEO', 'GSM', 'GSE', 'SM', 'GEI', 'MAG', 'SPH',
        'RLL', etc, combined with '-sph' or '-car'. E.g.

In [17]:
# Pull a real satellite trajectory
traj_dict, new_coord_sys = SF.SatelliteTrajectory('grace1', start_utcts, end_utcts, coord_type='GSE')
print(traj_dict.keys(), new_coord_sys)

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])
dict_keys(['sat_time', 'c1', 'c2', 'c3']) GSE-car


In [18]:
# Run ModelFlythrough with grace1 trajectory from SSCWeb
results = SF.ModelFlythrough(model, file_dir, variables, traj_dict['sat_time'], traj_dict['c1'], 
                             traj_dict['c2'], traj_dict['c3'], new_coord_sys)

Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 added from file.
Time slice index 0 added from file.
Time slice index 1 added from file.
Time slice index 2 added from file.
Time slice index 3 added from file.
Time slice index 4 added from file.
Time slice index 5 added from file.
Time slice index 6 added from file.
Time slice index 7 added from file.
Time slice index 8 added from file.
Time slice index 9 added from file.
Time slice index 10 added from file.
Time slice index 11 added from file.
Time slice index 12 added from file.
Time slice index 13 a

C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\scipy\interpolate\interpolate.py:630: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\rringuet\Anaconda3\envs\Kamodo_Jan2023\lib\site-packages\scipy\interpolate\interpolate.py:633: RuntimeWarning:

invalid value encountered in multiply



In [19]:
# Functionalize the output.
kamodo_object = SF.O.Functionalize_SFResults(model, results)
kamodo_object

{rho_n(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C45F8DEA68>, rho_n: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C45F8DEA68>, TEC(time): <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C45F8DECA8>, TEC: <function Functionalize_TimeSeries.<locals>.timeseries_func at 0x000001C45F8DECA8>}